<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [178]:
import pandas as pd
import psycopg2

In [179]:
connection_params = {"dbname":"project_sql",
                     "dbname":"project_sql",
                     "user":"skillfactory",
                     "password":"cCkxxLVrDE8EbvjueeMedPKt",
                     "host":"84.201.134.129",
                     "port":5432}

In [180]:
class DBObj():
    def __init__(self,connection_params=connection_params, query=None):
        # Создаем соединеине с БД
        connection = psycopg2.connect(**connection_params)
        # Создаем точку взаимодействия с БД - курсор
        cursor = connection.cursor()
        # Выполняем запрос к БД
        cursor.execute(query)
        # Возвращаем список строк
        self._records = cursor.fetchall()
        # Закроем соедиение
        connection.close()
    
    def to_string(self):
        '''Возвращает результат запроса в виде первой ячейки
        '''
        return str(self._records[0][0])

    def to_rows(self):
        '''Возвращает результат запроса в виде списка строк
        '''
        return self._records
    
    def to_table(self, columns=None):
        ''' Возвращает результат запроса в виде таблицы

        Arguments:
        columns - список столбцов
        '''
        return pd.DataFrame(self._records, columns=columns)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [181]:
query = f'''SELECT COUNT(*)
            FROM public.vacancies
            '''

In [182]:
# результат запроса
result_3_1 = f'Вакансий в базе: {DBObj(query=query).to_string()}'

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [183]:
# текст запроса
query = f'''SELECT COUNT(*)
            FROM public.employers
            '''

In [184]:
# результат запроса
result_3_2 = f'Работодателей в базе: {DBObj(query=query).to_string()}'

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [185]:
# текст запроса
query = f'''SELECT DISTINCT COUNT(*)
            FROM public.areas
            '''

In [186]:
# результат запроса
result_3_3 = f'Регионов в базе: {DBObj(query=query).to_string()}'

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [187]:
# текст запроса
query = f'''SELECT COUNT(*)
            FROM public.industries
            '''

In [188]:
# результат запроса
result_3_4 = f'Сфер деятельности в базе: {DBObj(query=query).to_string()}'

***

In [189]:
# выводы по предварительному анализу данных
print(f'Задание 3.1 {result_3_1}')
print(f'Задание 3.2 {result_3_2}')
print(f'Задание 3.3 {result_3_3}')
print(f'Задаине 3.4 {result_3_4}')


Задание 3.1 Вакансий в базе: 49197
Задание 3.2 Работодателей в базе: 23501
Задание 3.3 Регионов в базе: 1362
Задаине 3.4 Сфер деятельности в базе: 294


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [190]:
# текст запроса
query = f'''SELECT a.name as area,
            COUNT(v.name) as cnt
            FROM public.vacancies AS v
            JOIN
            public.areas AS a ON a.id=v.area_id
            GROUP BY a.name
            ORDER BY 2 DESC
            '''

In [191]:
# результат запроса
result_4_1 = DBObj(query=query).to_table(columns=["area", "cnt"])


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [192]:
# текст запроса
query = f'''SELECT COUNT(*)
            FROM public.vacancies AS v
            WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
            '''

In [193]:
# результат запроса
result_4_2 = f'Вакансии с хотя бы одним из двух полей вилки ЗП: {DBObj(query=query).to_string()}'


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [194]:
# текст запроса
query = f'''SELECT ROUND(AVG(v.salary_from)),
                   ROUND(AVG(v.salary_to))
            FROM public.vacancies AS v
            '''

In [195]:
# результат запроса
response = DBObj(query=query).to_rows()
result_4_3_1 = int(response[0][0])
result_4_3_2 = int(response[0][1])

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [196]:
# текст запроса
query = f'''SELECT v.schedule AS schedule,
                   v.employment AS employment,
                   COUNT(*)
            FROM public.vacancies AS v
            GROUP BY 1,2
            ORDER BY 3 DESC
            '''

In [197]:
# результат запроса
result_4_4 = DBObj(query=query).to_table(columns=["schedule", "employment", "qty"])

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [198]:
# текст запроса
query = f'''SELECT v.experience,
            COUNT(*)
            FROM public.vacancies AS v
            GROUP BY 1
            ORDER BY 2 ASC
            '''

In [199]:
# результат запроса
result_4_5 = DBObj(query=query).to_table(columns=["experience", "qty"])


***

In [200]:
# выводы по детальному анализу вакансий
print("Задание 4.1. Количество вакансий в каждом регионе:")
display(result_4_1)
print(f"Задание 4.2. Количесво вакансий с определенныйм диапазоном ЗП: {result_4_2}.")
print(f'Задание 4.3. Cредние значения для нижней и верхней границы зарплатной вилки: {result_4_3_1} руб - {result_4_3_2} руб.')
print("Задание 4.4. Количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment):")
display(result_4_4)
print("Задание 4.5. Количество вакансий с требуемым опытом работы (experience) по группам:")
display(result_4_5)


Задание 4.1. Количество вакансий в каждом регионе:


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


Задание 4.2. Количесво вакансий с определенныйм диапазоном ЗП: Вакансии с хотя бы одним из двух полей вилки ЗП: 24073.
Задание 4.3. Cредние значения для нижней и верхней границы зарплатной вилки: 71065 руб - 110537 руб.
Задание 4.4. Количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment):


,schedule,employment,qty
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


Задание 4.5. Количество вакансий с требуемым опытом работы (experience) по группам:


,experience,qty
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [201]:
# текст запроса
query = f'''SELECT e.name as emp,
            COUNT(v.id) as qty
            FROM public.vacancies AS v
            JOIN public.employers AS e ON e.id=v.employer_id
            GROUP BY e.name
            ORDER BY 2 DESC
            LIMIT 5
            '''
df = DBObj(query=query).to_table()

In [202]:
# результат запроса
result_5_1_1 = df.iloc[0][0]
result_5_1_2 = df.iloc[4][0]

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [203]:
# текст запроса
query = f'''WITH x (area_id, area, emp_qty) AS (SELECT 
                                                    a.id, 
                                                    a.name,
                                                    COUNT(emp.id)
            FROM public.areas AS a
            LEFT JOIN  public.employers AS emp ON a.id=emp.area
            GROUP BY a.id)
            
            SELECT temp.area_id,
                   temp.area,
                   temp.emp_qty,
                   COUNT(vac.id)
            FROM x as temp
            LEFT JOIN public.vacancies AS vac ON vac.area_id=temp.area_id
            GROUP BY temp.area_id, temp.area,temp.emp_qty
            ORDER BY COUNT(vac.id) ASC, temp.emp_qty DESC
         '''

In [204]:
# результат запроса
df = DBObj(query=query).to_table()
result_5_2 = df.iloc[0][1]

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [205]:
# текст запроса
query = f'''WITH x (area_id, emp_id)
            AS (SELECT DISTINCT vac.area_id,
                    emp.id
                FROM public.employers AS emp
                LEFT JOIN public.vacancies AS vac ON vac.employer_id=emp.id)
            SELECT
                    emp2.name,
                    COUNT(area_id)
            FROM x
            JOIN public.employers AS emp2 ON x.emp_id=emp2.id
            GROUP BY emp2.name
            ORDER BY 2 DESC
            '''

In [206]:
# результат запроса
result_5_3 = DBObj(query=query).to_table(columns=['работодатель', 'количество регионов с вакансиями'])

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [207]:
# текст запроса
query = f'''SELECT COUNT(*)
            FROM public.employers AS emp
            LEFT JOIN public.employers_industries AS ind ON ind.employer_id=emp.id
            WHERE ind.industry_id IS NULL
         '''

In [208]:
# результат запроса
result_5_4 = DBObj(query=query).to_string()

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [209]:
# текст запроса
query = f'''WITH x (emp, qty) AS (SELECT emp.name,
                                  COUNT(ind.industry_id)
                                  FROM public.employers AS emp
                                  LEFT JOIN public.employers_industries AS ind ON ind.employer_id=emp.id
                                  GROUP BY (emp.name))
            SELECT emp, qty
            FROM x
            WHERE x.qty=4
            ORDER BY emp
            OFFSET 2
            LIMIT 1
         '''

In [210]:
# результат запроса
result_5_5 = DBObj(query=query).to_string()

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [211]:
# текст запроса
query = f'''SELECT COUNT(*)
            FROM public.employers AS emp
            LEFT JOIN public.employers_industries AS ind ON ind.employer_id=emp.id
            JOIN public.industries AS ind_names ON ind_names.id=ind.industry_id
            WHERE ind_names.name='Разработка программного обеспечения'
         '''

In [212]:
# результат запроса
result_5_6 = DBObj(query=query).to_string()

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [213]:
# код для получения списка городов-милионников
from bs4 import BeautifulSoup 
import requests 

target_page = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

# Выполняем GET-запрос
response = requests.get(target_page)
# Получим html код
html_doc=response.text
soup = BeautifulSoup(html_doc, 'html.parser')
#Первая же таблица является искомым источником данных

table = soup.find('table')
table_body = table.find('tbody')
rows = table_body.find_all('tr')
# Создадим пустой список выходных данных
data = list()
# Получим данные в строках
for item in rows:
    row = item.find_all('td')
    row_data = list()
    #Получим данные из ячеек
    for cell in row:
        row_data.append(cell.text)
    #Если строка данных не пустая добавим их в выходной набор данных
    if row_data:
        # Попутно преобразуем данные в пригодный формат
        row_data[2] = int(row_data[2].replace(' ',''))*1000
        data.append(row_data[1:3])

# Проверим что получилось
city_data = pd.DataFrame(data, columns=['city', 'population'])
#display(city_data)
# Конвертируем список в строку для использования в SQL запросе
city_list_str = tuple(list(city_data['city']))

In [214]:
# текст запроса
query = f'''WITH x (ar, qty) AS (SELECT ar.name,
                                 COUNT(ar.name)
                                 FROM public.employers AS emp
                                 JOIN public.vacancies AS vac ON emp.id=vac.employer_id
                                 JOIN public.areas AS ar ON ar.id=vac.area_Id
                                 WHERE ar.name IN {city_list_str} AND emp.name='Яндекс'
                                 GROUP BY ar.name
                                 ORDER BY COUNT(ar.name))
            SELECT x.ar,
                   x.qty
            FROM x
                                 
            UNION
            
            SELECT 'Total',
                    SUM(x.qty)
            FROM x
            ORDER BY 2
         '''

In [215]:
# результат запроса
result_5_7 = DBObj(query=query).to_table(columns=['city','qty'])

***

In [216]:
# выводы по анализу работодателей
print(f'Задание 5.1. Работодатели на первом и пятом месте по количеству вакансий: {result_5_1_1}, {result_5_1_2}')
print(f'Задаине 5.2. Регион в котором нет вакансий, с наибольшим количеством работодателей: {result_5_2}')
print(f'Задание 5.3. Количество регионов, в которых работодатель публикует свои вакансии:')
display(result_5_3.loc[:10])
print(f'Задание 5.4. Количество работодателей, у которых не указана сфера деятельности: {result_5_4}')
print(f'Задание 5.5. Компания, на 3 месте (по названию), имеющая четыре сферы деятельности: {result_5_5}')
print(f'Задание 5.6. Количество работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»: {result_5_6}')
print(f'Задание 5.7. Количество вакансий "Яндекса" в регионах')
display(result_5_7)

Задание 5.1. Работодатели на первом и пятом месте по количеству вакансий: Яндекс, Газпром нефть
Задаине 5.2. Регион в котором нет вакансий, с наибольшим количеством работодателей: Россия
Задание 5.3. Количество регионов, в которых работодатель публикует свои вакансии:


,работодатель,количество регионов с вакансиями
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,КРОН,48
9,Почта России,48


Задание 5.4. Количество работодателей, у которых не указана сфера деятельности: 8419
Задание 5.5. Компания, на 3 месте (по названию), имеющая четыре сферы деятельности: 2ГИС
Задание 5.6. Количество работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»: 3553
Задание 5.7. Количество вакансий "Яндекса" в регионах


,city,qty
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [217]:
# текст запроса
query = f'''SELECT COUNT(vac.name)
            FROM public.vacancies AS vac
            WHERE LOWER(vac.name) LIKE '%data%' OR LOWER(vac.name) LIKE '%данн%'
            '''

In [218]:
# результат запроса
result_6_1 = DBObj(query=query).to_string()

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [219]:
# текст запроса
query = f'''WITH x (vac_name, vac_experience, vac_employment) AS 
            (SELECT vac.name, vac.experience, vac.employment
            FROM public.vacancies AS vac
            WHERE LOWER(vac.name) LIKE '%data%scientist%'
            OR LOWER(vac.name) LIKE '%data%science%'
            OR LOWER(vac.name) LIKE '%исследователь%данных%'
            OR LOWER(vac.name) LIKE '%ml%'
            AND LOWER(vac.name) NOT LIKE '%html%'
            OR LOWER(vac.name) LIKE '%machine%learning%'
            OR LOWER(vac.name) LIKE '%машинн%обучен%'
            )
            
            SELECT COUNT(*)
            FROM x
            WHERE LOWER(x.vac_name) LIKE '%junior%'
            OR LOWER(x.vac_experience) LIKE '%нет опыта%'
            OR LOWER(x.vac_employment) LIKE '%cтажировка%'
            '''

In [220]:
# результат запроса
result_6_2 = DBObj(query=query).to_string()

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [221]:
# текст запроса
query = f'''WITH x (vac_name, vac_key_skills) AS 
            (SELECT vac.name, vac.key_skills
            FROM public.vacancies AS vac
            WHERE LOWER(vac.name) LIKE '%data%scientist%'
            OR LOWER(vac.name) LIKE '%data%science%'
            OR LOWER(vac.name) LIKE '%исследователь%данных%'
            OR LOWER(vac.name) LIKE '%ml%'
            AND LOWER(vac.name) NOT LIKE '%html%'
            OR LOWER(vac.name) LIKE '%machine%learning%'
            OR LOWER(vac.name) LIKE '%машинн%обучен%'
            )
            SELECT COUNT(*)
            FROM x
            WHERE LOWER(x.vac_key_skills) LIKE ('%sql%')
            OR LOWER(x.vac_key_skills) LIKE ('%postgres%')
        '''

In [222]:
# результат запроса
result_6_3 = DBObj(query=query).to_string()

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [223]:
# текст запроса
query = f'''WITH x (vac_name, vac_key_skills) AS 
            (SELECT vac.name, vac.key_skills
            FROM public.vacancies AS vac
            WHERE LOWER(vac.name) LIKE '%data%scientist%'
            OR LOWER(vac.name) LIKE '%data%science%'
            OR LOWER(vac.name) LIKE '%исследователь%данных%'
            OR LOWER(vac.name) LIKE '%ml%'
            AND LOWER(vac.name) NOT LIKE '%html%'
            OR LOWER(vac.name) LIKE '%machine%learning%'
            OR LOWER(vac.name) LIKE '%машинн%обучен%'
            )
            SELECT COUNT(*)
            FROM x
            WHERE LOWER(x.vac_key_skills) LIKE ('%python%')
        '''

In [224]:
# результат запроса
result_6_4 = DBObj(query=query).to_string()

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [225]:
# текст запроса
query = f'''select
            round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1),2)
            
            from vacancies v

            WHERE (v.name ILIKE '%data scientist%')
            OR (lower(v.name) ILIKE '%data science%')
            OR (v.name ILIKE '%исследователь данных%')
            OR (v.name  LIKE '%ML%' AND  (v.name NOT iLIKE '%html%'))
            OR (v.name ILIKE '%machine learning%')
            OR (v.name ILIKE '%машинн%обучен%')
            AND (key_skills IS NOT NULL)
        '''

In [226]:
# результат запроса
result_6_5 = DBObj(query=query).to_string()

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [227]:
query = f'''WITH x (exp, s_from, s_to) AS
            (
            SELECT vac.experience, vac.salary_from, vac.salary_to
            FROM public.vacancies AS vac
            
            WHERE (vac.name ILIKE '%data scientist%')
            OR (LOWER(vac.name) ILIKE '%data science%')
            OR (vac.name ILIKE '%исследователь данных%')
            OR (vac.name LIKE '%ML%' AND (vac.name NOT ILIKE '%HTML%'))
            OR (vac.name ILIKE '%machine learning%')
            OR (vac.name ILIKE '%машинн%обучен%')
            )

            SELECT
            ROUND(AVG(COALESCE((s_to + s_from)/2, s_from, s_to)), 0)
            FROM x
            WHERE (s_from IS NOT NULL OR s_to IS NOT NULL)
            AND (x.exp ILIKE '%3 до 6%')
            GROUP BY x.exp
        '''

In [228]:
# результат запроса
result_6_6 = DBObj(query=query).to_string()

***

In [229]:
# выводы по предметному анализу
print(f'Задание 6.1. Количество вакансий имеющих отношение к данным: {result_6_1}')
print(f'Задание 6.2. Количиство вакансий для начинающего специалиста по анализу данных: {result_6_2}')
print(f'Задание 6.3. Количество вакансий где требуются навыки SQL и postgeres: {result_6_3}')
print(f'Задание 6.4. Количество вакансий где требуются навыки Python: {result_6_4}')
print(f'Задание 6.5. Среднее количество ключевых навыков в вакансиях для DS: {result_6_5}')
print(f'Задание 6.6. Средняя ЗП специалиста по анализу данных с опытом работы от 3 до 6 лет: {result_6_6} руб.')

Задание 6.1. Количество вакансий имеющих отношение к данным: 1771
Задание 6.2. Количиство вакансий для начинающего специалиста по анализу данных: 51
Задание 6.3. Количество вакансий где требуются навыки SQL и postgeres: 229
Задание 6.4. Количество вакансий где требуются навыки Python: 357
Задание 6.5. Среднее количество ключевых навыков в вакансиях для DS: 6.41
Задание 6.6. Средняя ЗП специалиста по анализу данных с опытом работы от 3 до 6 лет: 243115 руб.


# Общий вывод по проекту

В ходе выполнения заданий было выявлено:
* База охватывает информацию о вакансиях в большинстве регионов РФ и ближнего зарубежья в различных сферах деятельности
* Большая часть вакансий так или иначе пересекается с IT сферой
* В базе имееются пропуски или искажения, которые препятсвуют формированию прогнозов и более точной оценки "стоимости специалиста"
      например, зарплатная вилка или слишком большая или неуказана вовсе, многие организации не указали сфер деятельности.
Далее проведем небольшую модификацию данных что бы придать больше смысла сбранным данным.

In [230]:
'''Расширить исследование возможно при помощи разделения вакансий по категориям.
Из наиболее встречающихся слов в названиях вакансий составим список категорий подхлодящих под описание:
* специалист широкого профиля, 
* специалист узкого профиля
* руководитель
'''

query = f'''SELECT vac.name
            FROM public.vacancies AS vac
            '''
rows = DBObj(query=query).to_rows()

def clear_string(string):
    non_alpha_char = ['.',',','-','/','\\','(',')','\t','\n']
    for char in string:
        if char in non_alpha_char:
            string = string.replace(char, '*')
    return string.lower()

word_dict = dict()
for row in rows:
    word_list = row[0].split()
    for word in word_list:
        word = clear_string(word)
        if word not in word_dict and len(word)>1:
            word_dict[word] = 1
        elif len(word)>2:
            word_dict[word] += 1

word_data = [(k, v) for k, v in word_dict.items()]

df = pd.DataFrame(word_data, columns=['word', 'qty'])
df = df.sort_values(by=['qty'], ascending=False)
df = df[df['qty']>10]
#display(list(df['word'].iloc[399:500]))


In [231]:
'''Из текста полученного выше выделим слова, которые могут косвенно соотнести вакансию с категорией должности
Добавим признаки типа должности к вакансиям в запрос.
'''

input_cols = f"id, name, key_skills, schedule, experience, employment, salary_from, salary_to, area_id, employer_id"
ouput_cols = f"id, name, key_skills, schedule, experience, employment, salary_from, salary_to, area_id, employer_id, type"
big_query = f'''WITH 
            x1 ({ouput_cols}) AS

            (
            SELECT {input_cols}, 'wide_specialist' AS type
            
            FROM public.vacancies AS vac
            WHERE (vac.name ILIKE '%manag%' AND (vac.name NOT ILIKE '%proj%manag%') AND (vac.name NOT ILIKE '%prod%manag%'))
            OR(vac.name ILIKE '%менедж%' AND (vac.name NOT ILIKE '%прод%') AND (vac.name NOT ILIKE '%проект%'))
            OR(vac.name ILIKE '%админ%' AND (vac.name NOT ILIKE '%старш%') AND (vac.name NOT ILIKE '%инжен%'))
            OR(vac.name ILIKE '%спец%' AND (vac.name NOT ILIKE '%ведущий спец%')AND (vac.name NOT ILIKE '%главный спец%'))
            OR(vac.name ILIKE '%тестировщик%' AND (vac.name NOT ILIKE '%ведущ%') AND (vac.name NOT ILIKE '%старш%'))
            OR(vac.name ILIKE '%консул%')
            OR(vac.name ILIKE '%аналит%')
            OR(vac.name ILIKE '%ученик%')
            OR(vac.name ILIKE '%маркетолог%')
            OR(vac.name ILIKE '%cтажер%')
            OR(vac.name ILIKE '%junior%')
            OR(vac.name ILIKE '%младший%')
            OR(vac.name ILIKE '%сотрудник%')
            OR(vac.name ILIKE '%оператор%')
            OR(vac.name ILIKE '%support%')
            OR(vac.name ILIKE '%начинающий%')
            OR(vac.name ILIKE '%координатор%')
            OR(vac.name ILIKE '%admin%')
            OR(vac.name ILIKE '%корректор%')
            )

            ,x2 ({ouput_cols}) AS
            
            (
            SELECT 
            {input_cols}, 'narrow_specialist' AS type
            
            FROM public.vacancies AS vac
            WHERE (vac.name ILIKE '%develop%')
            OR(vac.name ILIKE '%senior%')
            OR(vac.name ILIKE '%engin%')
            OR(vac.name ILIKE '%программист%' AND (vac.name NOT ILIKE '%cтажер%'))
            OR(vac.name ILIKE '%разраб%' AND (vac.name NOT ILIKE '%cтажер%'))
            OR(vac.name ILIKE '%инжен%')
            OR(vac.name ILIKE '%эксперт%')
            OR(vac.name ILIKE '%middl%')
            OR(vac.name ILIKE '%ведущий специалист%')
            OR(vac.name ILIKE '%старш%')
            OR(vac.name ILIKE '%мастер%')
            OR(vac.name ILIKE '%automation%')
            OR(vac.name ILIKE '%scientist%')
            OR(vac.name ILIKE '%архитектор%')
            OR(vac.name ILIKE '%заместитель%')
            OR(vac.name ILIKE '%преподаватель%')
            OR(vac.name ILIKE '%верстал%')
            OR(vac.name ILIKE '%машинист%')
            OR(vac.name ILIKE '%методолог%')
            OR(vac.name ILIKE '%officer%')
            OR(vac.name ILIKE '%главный%' AND (vac.name ILIKE '%специалист%'))
            OR(vac.name ILIKE '%manag%' AND (vac.name ILIKE '%proj%manag%') AND (vac.name ILIKE '%prod%manag%'))
            OR(vac.name ILIKE '%менед%' AND (vac.name ILIKE '%прод%') AND (vac.name ILIKE '%проек%'))
            )
            
            ,x3 ({ouput_cols}) AS
            (
            SELECT 
            {input_cols}, 'head' AS type

            FROM public.vacancies AS vac
            WHERE (vac.name ILIKE '%руковод%')
            OR(vac.name ILIKE '%lead%')

            OR(vac.name ILIKE '%начальник%')
            OR(vac.name ILIKE '%директор%')
            OR(vac.name ILIKE '%тимлид%')
            OR(vac.name ILIKE '%chief%')
            )

            ,x4 ({ouput_cols}) AS
            (
            SELECT *
            FROM 
            x1

            UNION ALL

            SELECT *
            FROM 
            x2

            UNION ALL

            SELECT *
            FROM 
            x3)

            --SELECT x4.id, x4.name, x4.type, x4.key_skills, x4.schedule, x4.experience, x4.employment, x4.salary_from, x4.salary_to, x4.area_id, ar.name, x4.employer_id,  emp.name
            
            SELECT x4.id, 
                   x4.name,
                   x4.type,
                   x4.key_skills,
                   x4.schedule,
                   x4.experience,
                   x4.employment,
                   ROUND(AVG(COALESCE((x4.salary_from + x4.salary_to)/2, x4.salary_from,  x4.salary_to)), 0),
                   x4.area_id,
                   ar.name,
                   x4.employer_id,
                   emp.name
                   
            
            
            FROM x4
            
            JOIN public.employers AS emp ON emp.id=x4.employer_id
            JOIN public.areas AS ar ON ar.id=x4.area_id
            GROUP BY x4.id, 
                   x4.name,
                   x4.type,
                   x4.key_skills,
                   x4.schedule,
                   x4.experience,
                   x4.employment,
                   x4.area_id,
                   ar.name,
                   x4.employer_id,
                   emp.name


        '''
df = DBObj(query=big_query).to_table(columns=['vac_id','vac_name', 'vac_type','vac_slills','schedule','experience','employment','salary','area_id','area_name', 'empl_id', 'emp_name'])

In [232]:
print('''Признаки имеют некоторое расхождение в единичных случаях с описанием вакансии, но с помощью них можно подсчитать процентное соотношений вакансий, получаемую ЗП, опыт работы необходимый для категорий должностей.
Далее в каких организациях требуются, какие категории сотрудников, соответвенно сделать вывод дефицит каких работников наблюдается в той или иной организации и т.д.
''')
import plotly
import plotly.graph_objs as go
import plotly.express as px

head_count = df['vac_id'][df['vac_type']=='head'].count()
wide_specialist_count = df['vac_id'][df['vac_type']=='wide_specialist'].count()
narrow_specialist_count = df['vac_id'][df['vac_type']=='narrow_specialist'].count()
types = ['head', 'wide_specialist','narrow_specialist']
data = list(zip(types,[head_count, wide_specialist_count, narrow_specialist_count]))
pie_df = pd.DataFrame(data, columns=['type', 'qty'])
display(pie_df)

fig = go.Figure()
fig.add_trace(go.Pie(values=pie_df['qty'], 
                     labels=['Руководящая должность','Специалист широкого профиля','Специалист узкого профиля'],
                     hovertemplate= '%{label} %{percent}'+'<extra></extra>'))

fig.update_layout(autosize=False, width=550, height=400,
                  title="Распределение вакансий по категориям на рынке труда",
                  legend_title="Категории:",
                  legend_orientation="h",
                  title_x = 0.5)
fig.write_html('_includes/1.html')
fig.write_image('_includes/1.png')
fig.show()

Признаки имеют некоторое расхождение в единичных случаях с описанием вакансии, но с помощью них можно подсчитать процентное соотношений вакансий, получаемую ЗП, опыт работы необходимый для категорий должностей.
Далее в каких организациях требуются, какие категории сотрудников, соответвенно сделать вывод дефицит каких работников наблюдается в той или иной организации и т.д.



,type,qty
0,head,3752
1,wide_specialist,19157
2,narrow_specialist,25692


![Рис. 1](_includes/1.png)

Промежуточный вывод: Наиболее востребованы узконаправленные специалисты. Проверим насколько востребованность соотносится с предлагаемой заработной платой.

In [233]:
print('''Подсчитаем среднюю ЗП для разных категорий вакансий.
Проблема заключается в том что для большинства вакансий не указана ЗП
Поэтому, придется для начала выяснить процент пропусков, а затем реализовать механизм компенсации недостающих данных.
''')
temp = df.copy()

def show_non_salary_vacancies(input_df):
    '''Выводит на экран данные по пропускам значений ЗП в наборе данных
    '''
    search_df = input_df.copy()
    try:
       search_df.drop(['vac_name','vac_slills','schedule','experience','employment', 'area_id', 'area_name', 'empl_id', 'emp_name'], axis=1, inplace=True)
    except:
        pass
    search_df = search_df[search_df['salary'].isna()]
    narrow = search_df[search_df['vac_type']=='narrow_specialist'].shape[0]
    head = search_df[search_df['vac_type']=='head'].shape[0]
    wide = search_df[search_df['vac_type']=='wide_specialist'].shape[0]
    total_vac = input_df.shape[0]
    print(f"Количетво вакансий узких специалистов без ЗП: {narrow}")
    print(f"Количетво руководящих должностей без ЗП: {head}")
    print(f"Количество специалистов широкого профиля без ЗП: {wide}")
    print(f"Общее количество вакансий: {total_vac}")
    print(f"Процент пропусков {round(((narrow+head+wide)/total_vac)*100, 2)} %")
    
show_non_salary_vacancies(temp)


Подсчитаем среднюю ЗП для разных категорий вакансий.
Проблема заключается в том что для большинства вакансий не указана ЗП
Поэтому, придется для начала выяснить процент пропусков, а затем реализовать механизм компенсации недостающих данных.

Количетво вакансий узких специалистов без ЗП: 13968
Количетво руководящих должностей без ЗП: 2125
Количество специалистов широкого профиля без ЗП: 8767
Общее количество вакансий: 48601
Процент пропусков 51.15 %


In [234]:

print ('''За ориентир предполагаемой средней ЗП для разных категорий можно взять вакансии крупных работодателей.
Например, из результатов полученных выше такими ориентирами могут быть "Яндекс" "Ростелеком" "Спецремонт" и т.д.''')
print()
df['empl_id'] = df['empl_id'].astype('int64')
big_employers_df = df[df['emp_name'].str.contains('Янд') 
                      | df['emp_name'].str.contains('Почта') 
                      | df['emp_name'].str.contains('Спецремонт') 
                      | df['emp_name'].str.contains('ЕФИН')
                      | df['emp_name'].str.contains('Поляков Денис Иванович')
                      | df['emp_name'].str.contains('Красное & Белое')
                      | df['emp_name'].str.contains('Тензор')
                      ]

show_non_salary_vacancies(big_employers_df)



За ориентир предполагаемой средней ЗП для разных категорий можно взять вакансии крупных работодателей.
Например, из результатов полученных выше такими ориентирами могут быть "Яндекс" "Ростелеком" "Спецремонт" и т.д.

Количетво вакансий узких специалистов без ЗП: 125
Количетво руководящих должностей без ЗП: 20
Количество специалистов широкого профиля без ЗП: 258
Общее количество вакансий: 2569
Процент пропусков 15.69 %


In [235]:
print('''Итак, основываясь на более подробной выборке сделаем прогнозы по средней заработной плате для разных категорий
и добавим их в исходный набор
''')

def show_mean_sallary(input_df):
    search_df = input_df.copy()
    narrow = search_df[search_df['vac_type']=='narrow_specialist']['salary'].mean()
    head = search_df[search_df['vac_type']=='head']['salary'].mean()
    wide = search_df[search_df['vac_type']=='wide_specialist']['salary'].mean()
    print(f"Средняя ЗП узкого специалиста: {round(narrow)} руб.")
    print(f"Средняя ЗП руководителя: {round(head)} руб.")
    print(f"Средняя ЗП специалиста широкого профиля: {round(wide)} руб.")


show_mean_sallary(big_employers_df)

Итак, основываясь на более подробной выборке сделаем прогнозы по средней заработной плате для разных категорий
и добавим их в исходный набор

Средняя ЗП узкого специалиста: 95580 руб.
Средняя ЗП руководителя: 109935 руб.
Средняя ЗП специалиста широкого профиля: 26791 руб.


In [236]:
#Уберем предупреждение о том что будут перезаписаны данные в pandas DataFrame
pd.options.mode.chained_assignment = None 

new_df_head = temp.copy()
new_df_narrow = temp.copy()
new_df_wide = temp.copy()

new_df_narrow = temp[temp['vac_type']=='narrow_specialist']
new_df_narrow['salary'] = new_df_narrow['salary'].fillna(95580)

new_df_head = temp[temp['vac_type']=='head']
new_df_head['salary'] = new_df_head['salary'].fillna(109935)

new_df_wide = temp[temp['vac_type']=='wide_specialist']
new_df_wide['salary'] = new_df_wide['salary'].fillna(26791)

combined = pd.concat([new_df_narrow, new_df_head, new_df_wide])
print('''Проверим, что пропусков не осталось и какие значения у нас получились...
      ''')
show_non_salary_vacancies(combined)
print()
show_mean_sallary(combined)




Проверим, что пропусков не осталось и какие значения у нас получились...
      
Количетво вакансий узких специалистов без ЗП: 0
Количетво руководящих должностей без ЗП: 0
Количество специалистов широкого профиля без ЗП: 0
Общее количество вакансий: 48601
Процент пропусков 0.0 %

Средняя ЗП узкого специалиста: 99855 руб.
Средняя ЗП руководителя: 115314 руб.
Средняя ЗП специалиста широкого профиля: 43318 руб.


Промежуточный вывод: востребованность узконаправенных высоквалифицированных специалистов подтвердилась 
предлагаемой заработной платой. Она немногим ниже чем средняя у руководящих должностей.


Далее нужно построить столбчатую диаграмму. Также можно проиндексировать данные для того что бы получить примерный данные на сегодняшний день...

Возмем за отправную точку 2020 год. Согласно [данным об инфляции](https://xn----ctbjnaatncev9av3a8f8b.xn--p1ai/%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8B-%D0%B8%D0%BD%D1%84%D0%BB%D1%8F%D1%86%D0%B8%D0%B8)
получим следующую таблицу

|       2020|       2021|       2022|       2023|       2024|
|-----------|-----------|-----------|-----------|-----------|
|      4.91%|      8.39%|     11.92%|      7.42%|         9%|


*т.к. 2024 еще не закончился условно примем за 9%

In [237]:
bar_data = pd.DataFrame({
    'Group': ['wide_specialist', 'narrow_specialist', 'head'],
    '2020': [round(combined[combined['vac_type']=='wide_specialist']['salary'].mean()),
             round(combined[combined['vac_type']=='narrow_specialist']['salary'].mean()),
             round(combined[combined['vac_type']=='head']['salary'].mean())]
    })
year = [2020,2021,2022,2023,2024,2025]
inflation = [3.05,4.91,8.39,11.92,7.42,9]

for i in range(1,len(inflation)):
     
      bar_data[str(year[i])]=round((bar_data.iloc[:,[i]])/(1-(inflation[i])/100))

print('''Получаем данные по средней заработной плате за несколько лет: 
''')
display(bar_data)


Получаем данные по средней заработной плате за несколько лет: 



,Group,2020,2021,2022,2023,2024,2025
0,wide_specialist,43318,45555.0,49727.0,56457.0,60982.0,67013.0
1,narrow_specialist,99855,105011.0,114628.0,130141.0,140571.0,154474.0
2,head,115314,121268.0,132374.0,150288.0,162333.0,178388.0


In [238]:
'''Отобразим результаты на столбчатой диаграмме'''

#строим график
data=[go.Bar(x=bar_data['Group'],
               y=bar_data['2020'], #датафрейм
               name='2020',
               hovertemplate= 'Категория: %{x}<br>'+'ЗП: %{customdata}'+'<extra></extra>',
               customdata=list(map(lambda x: f'{round((x/1000),2)} тыс.', list(bar_data['2020'])))
               ),
    go.Bar(x=bar_data['Group'],
           y=bar_data['2021'], #датафрейм
               name='2021',
               hovertemplate= 'Категория: %{x}<br>'+'ЗП: %{customdata}'+'<extra></extra>',
               customdata=list(map(lambda x: f'{round((x/1000),2)} тыс.', list(bar_data['2021'])))
               ),
    go.Bar(x=bar_data['Group'],
           y=bar_data['2022'], #датафрейм
               name='2022',
               hovertemplate= 'Категория: %{x}<br>'+'ЗП: %{customdata}'+'<extra></extra>',
               customdata=list(map(lambda x: f'{round((x/1000),2)} тыс.', list(bar_data['2022'])))
               ),
    go.Bar(x=bar_data['Group'],
           y=bar_data['2023'], #датафрейм
               name='2023',
               hovertemplate= 'Категория: %{x}<br>'+'ЗП: %{customdata}'+'<extra></extra>',
               customdata=list(map(lambda x: f'{round((x/1000),2)} тыс.', list(bar_data['2023'])))
               ),
    go.Bar(x=bar_data['Group'],
           y=bar_data['2024'], #датафрейм
               name='2024',
               hovertemplate= 'Категория: %{x}<br>'+'ЗП: %{customdata}'+'<extra></extra>',
               customdata=list(map(lambda x: f'{round((x/1000),2)} тыс.', list(bar_data['2024'])))
               ),
    go.Bar(x=bar_data['Group'],
           y=bar_data['2025'], #датафрейм
               name='2025',
               hovertemplate= 'Категория: %{x}<br>'+'ЗП: %{customdata}'+'<extra></extra>',
               customdata=list(map(lambda x: f'{round((x/1000),2)} тыс.', list(bar_data['2025'])))
               )
       ]

fig = go.Figure(data)

fig.update_layout(autosize=False, width=750, height=400,
                  legend_title='Год',
                  xaxis_title="Категории вакансий",
                  yaxis_title="Средняя ожидаемая ЗП",
                  xaxis = {'tickmode': 'array',
                            'tickvals':list(bar_data['Group']),
                            'ticktext': ['специалист широкого профиля','специалист узкого профиля','руководитель']},
                  yaxis = {'tickmode': 'array',
                           'tickvals': list(range(20000,220000,20000)),
                           'ticktext': list(map(lambda x: f'{int(x/1000)} тыс.', list(range(20000,220000,20000))))
                           })
fig.write_html('_includes/2.html')
fig.write_image('_includes/2.png')
fig.show()

![Рис. 2](_includes/2.png)

В конечном итоге мы получили ожидаемую среднюю заработную плату в 2025 году для разных категорий вакансий исходя из анализа рынка труда и инфляционных ожиданий.

### Спасибо за внимание.